## Importing necessary libraries

In [1]:
import cv2
import time
import numpy as np
from random import randint
import argparse
import sys
import time
%run "C:/Users/birud/OneDrive - iitgn.ac.in/github/openpose_clone/python/import_pyopenpose.py"
print("pyopenpose imported")

pyopenpose imported


## Defining some variables

In [2]:
threshold = 0.2
#Body_25 model use 25 points
key_points = {
    0:  "Nose", 1:  "Neck", 2:  "RShoulder", 3:  "RElbow", 4:  "RWrist", 5:  "LShoulder", 6:  "LElbow",
    7:  "LWrist", 8:  "MidHip", 9:  "RHip", 10: "RKnee", 11: "RAnkle", 12: "LHip", 13: "LKnee",
    14: "LAnkle", 15: "REye", 16: "LEye", 17: "REar", 18: "LEar", 19: "LBigToe", 20: "LSmallToe",
    21: "LHeel", 22: "RBigToe", 23: "RSmallToe", 24: "RHeel", 25: "Background"
}

#Body_25 keypoint pairs 
POSE_PAIRS = [[1,2], [1,5], [2,3], [3,4], [5,6], [6,7],     #arm, shoulder line
            [1,8], [8,9], [9,10], [10,11], [8,12], [12,13], [13,14],  #2 leg
            [11,24], [11,22], [22,23], [14,21],[14,19],[19,20],    #2 foot  
            [1,0], [0,15], [15,17], [0,16], [16,18], #face
#             [2,17], [5,18]
            ]  

colors = [ [0,100,255], [0,100,255], [0,255,255], [0,100,255], [0,255,255], [0,100,255],
         [0,255,0], [255,200,100], [255,0,255], [0,255,0], [255,200,100], [255,0,255],
         [0,0,255], [255,0,0], [200,200,0], [255,0,0], [200,200,0], [0,0,0]]

nPoints = 25

## Importing the openpose model from the local system

In [3]:
protoFile = r"C:\Users\birud\OneDrive - iitgn.ac.in\github\openpose_clone\models\pose\body_25\pose_deploy.prototxt"
weightsFile = r"C:\Users\birud\OneDrive - iitgn.ac.in\github\openpose_clone\models\pose\body_25\pose_iter_584000.caffemodel"

## Getting the frame rate of the video

In [4]:
i = input("Enter the video path: ")
args.video = r"%s" % i
cap = cv2.VideoCapture(args.video)
ret, img = cap.read()
if ret == False:
    print('Video File Read Error')
    sys.exit(0)
frameHeight, frameWidth, c = img.shape

# Get the frames per second (fps) of the video
fps = cap.get(cv2.CAP_PROP_FPS)

print(f"Frame rate of the video: {fps} frames per second")

Enter the video path: C:\Users\birud\OneDrive - iitgn.ac.in\github\openpose_clone\examples\media\video3.mp4
Frame rate of the video: 200.0 frames per second


In [5]:
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
out_video_path = r'C:\Users\birud\OneDrive - iitgn.ac.in\github\Pose-estimation\files\OpenPose\processed video openpose\{}-{}-output2.mp4'.format(args.model, args.device)
out_video = cv2.VideoWriter(out_video_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (frameWidth, frameHeight))

## The Below Code applies the model for each frame

In [6]:
frame = 0
inHeight = 368
params = dict()
params["model_folder"] = r"C:\Users\birud\OneDrive - iitgn.ac.in\github\openpose_clone\models"
params["net_resolution"] = "368x-1"
params["display"] = "0"
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()
t_elapsed = 0.0

# Set initial values for thickness
line_thickness = 2
circle_thickness = 2

while cap.isOpened():
    f_st = time.time()
    ret, img = cap.read()
    if ret == False:
        break
    frame += 1
    datum = op.Datum()
    datum.cvInputData = img
    opWrapper.emplaceAndPop(op.VectorDatum([datum]))

    # Check if poseKeypoints is not None before accessing its length
    if datum.poseKeypoints is not None:
        human_count = len(datum.poseKeypoints)
    else:
        human_count = 0

    frameClone = img.copy()
    for human in range(human_count):
        for j in range(nPoints):
            if datum.poseKeypoints[human][j][2] > threshold:
                circle_center = tuple(map(int, datum.poseKeypoints[human][j][0:2]))
                cv2.circle(frameClone, circle_center, circle_thickness, colors[j % 17], -1, cv2.LINE_AA)

    for human in range(human_count):
        for pair in POSE_PAIRS:
            if datum.poseKeypoints[human][pair[0]][2] <= threshold or datum.poseKeypoints[human][pair[1]][2] <= threshold:
                continue
            S = (int(datum.poseKeypoints[human][pair[0]][0]),
                 int(datum.poseKeypoints[human][pair[0]][1]))
            E = (int(datum.poseKeypoints[human][pair[1]][0]),
                 int(datum.poseKeypoints[human][pair[1]][1]))
            cv2.line(frameClone, S, E, colors[pair[0] % 17], line_thickness, cv2.LINE_AA)

    out_video.write(frameClone)
    f_elapsed = time.time() - f_st
    t_elapsed += f_elapsed
    print('Frame[%d] processed time[%4.2f]' % (frame, f_elapsed))

print('Total processed time[%4.2f]' % (t_elapsed))
print('avg frame processing rate :%4.2f' % (t_elapsed / frame))
cap.release()
out_video.release()

Frame[1] processed time[2.27]
Frame[2] processed time[0.13]
Frame[3] processed time[0.12]
Frame[4] processed time[0.12]
Frame[5] processed time[0.13]
Frame[6] processed time[0.12]
Frame[7] processed time[0.12]
Frame[8] processed time[0.11]
Frame[9] processed time[0.12]
Frame[10] processed time[0.12]
Frame[11] processed time[0.12]
Frame[12] processed time[0.10]
Frame[13] processed time[0.11]
Frame[14] processed time[0.10]
Frame[15] processed time[0.12]
Frame[16] processed time[0.12]
Frame[17] processed time[0.12]
Frame[18] processed time[0.12]
Frame[19] processed time[0.11]
Frame[20] processed time[0.12]
Frame[21] processed time[0.11]
Frame[22] processed time[0.12]
Frame[23] processed time[0.12]
Frame[24] processed time[0.10]
Frame[25] processed time[0.09]
Frame[26] processed time[0.10]
Frame[27] processed time[0.10]
Frame[28] processed time[0.10]
Frame[29] processed time[0.10]
Frame[30] processed time[0.09]
Frame[31] processed time[0.12]
Frame[32] processed time[0.11]
Frame[33] process

Frame[262] processed time[0.12]
Frame[263] processed time[0.12]
Frame[264] processed time[0.12]
Frame[265] processed time[0.09]
Frame[266] processed time[0.11]
Frame[267] processed time[0.12]
Frame[268] processed time[0.10]
Frame[269] processed time[0.10]
Frame[270] processed time[0.10]
Frame[271] processed time[0.09]
Frame[272] processed time[0.10]
Frame[273] processed time[0.10]
Frame[274] processed time[0.10]
Frame[275] processed time[0.12]
Frame[276] processed time[0.12]
Frame[277] processed time[0.09]
Frame[278] processed time[0.10]
Frame[279] processed time[0.11]
Frame[280] processed time[0.10]
Frame[281] processed time[0.11]
Frame[282] processed time[0.10]
Frame[283] processed time[0.11]
Frame[284] processed time[0.09]
Frame[285] processed time[0.12]
Frame[286] processed time[0.11]
Frame[287] processed time[0.12]
Frame[288] processed time[0.10]
Frame[289] processed time[0.10]
Frame[290] processed time[0.10]
Frame[291] processed time[0.11]
Frame[292] processed time[0.11]
Frame[29

Frame[520] processed time[0.11]
Frame[521] processed time[0.11]
Frame[522] processed time[0.12]
Frame[523] processed time[0.09]
Frame[524] processed time[0.11]
Frame[525] processed time[0.12]
Frame[526] processed time[0.11]
Frame[527] processed time[0.12]
Frame[528] processed time[0.12]
Frame[529] processed time[0.11]
Frame[530] processed time[0.11]
Frame[531] processed time[0.13]
Frame[532] processed time[0.12]
Frame[533] processed time[0.12]
Frame[534] processed time[0.12]
Frame[535] processed time[0.12]
Frame[536] processed time[0.12]
Frame[537] processed time[0.12]
Frame[538] processed time[0.10]
Frame[539] processed time[0.12]
Frame[540] processed time[0.12]
Frame[541] processed time[0.11]
Frame[542] processed time[0.09]
Frame[543] processed time[0.12]
Frame[544] processed time[0.08]
Frame[545] processed time[0.11]
Frame[546] processed time[0.12]
Frame[547] processed time[0.09]
Frame[548] processed time[0.11]
Frame[549] processed time[0.11]
Frame[550] processed time[0.10]
Frame[55

Frame[777] processed time[0.11]
Frame[778] processed time[0.09]
Frame[779] processed time[0.10]
Frame[780] processed time[0.10]
Frame[781] processed time[0.09]
Frame[782] processed time[0.10]
Frame[783] processed time[0.10]
Frame[784] processed time[0.10]
Frame[785] processed time[0.10]
Frame[786] processed time[0.09]
Frame[787] processed time[0.12]
Frame[788] processed time[0.10]
Frame[789] processed time[0.10]
Frame[790] processed time[0.11]
Frame[791] processed time[0.10]
Frame[792] processed time[0.11]
Frame[793] processed time[0.09]
Frame[794] processed time[0.12]
Frame[795] processed time[0.10]
Frame[796] processed time[0.11]
Frame[797] processed time[0.10]
Frame[798] processed time[0.11]
Frame[799] processed time[0.09]
Frame[800] processed time[0.11]
Frame[801] processed time[0.12]
Frame[802] processed time[0.12]
Frame[803] processed time[0.12]
Frame[804] processed time[0.11]
Frame[805] processed time[0.10]
Frame[806] processed time[0.12]
Frame[807] processed time[0.12]
Frame[80

Frame[1034] processed time[0.11]
Frame[1035] processed time[0.10]
Frame[1036] processed time[0.12]
Frame[1037] processed time[0.12]
Frame[1038] processed time[0.13]
Frame[1039] processed time[0.12]
Frame[1040] processed time[0.12]
Frame[1041] processed time[0.12]
Frame[1042] processed time[0.11]
Frame[1043] processed time[0.12]
Frame[1044] processed time[0.11]
Frame[1045] processed time[0.11]
Frame[1046] processed time[0.12]
Frame[1047] processed time[0.11]
Frame[1048] processed time[0.12]
Frame[1049] processed time[0.12]
Frame[1050] processed time[0.12]
Frame[1051] processed time[0.11]
Frame[1052] processed time[0.12]
Frame[1053] processed time[0.13]
Frame[1054] processed time[0.13]
Frame[1055] processed time[0.11]
Frame[1056] processed time[0.11]
Frame[1057] processed time[0.09]
Frame[1058] processed time[0.11]
Frame[1059] processed time[0.10]
Frame[1060] processed time[0.11]
Frame[1061] processed time[0.11]
Frame[1062] processed time[0.12]
Frame[1063] processed time[0.11]
Frame[1064

...

Frame[6026] processed time[0.14]
Frame[6027] processed time[0.09]
Frame[6028] processed time[0.11]
Frame[6029] processed time[0.10]
Frame[6030] processed time[0.11]
Frame[6031] processed time[0.11]
Frame[6032] processed time[0.10]
Frame[6033] processed time[0.09]
Frame[6034] processed time[0.11]
Frame[6035] processed time[0.10]
Frame[6036] processed time[0.10]
Frame[6037] processed time[0.09]
Frame[6038] processed time[0.10]
Frame[6039] processed time[0.10]
Frame[6040] processed time[0.10]
Frame[6041] processed time[0.12]
Frame[6042] processed time[0.12]
Frame[6043] processed time[0.12]
Frame[6044] processed time[0.10]
Frame[6045] processed time[0.11]
Frame[6046] processed time[0.10]
Frame[6047] processed time[0.13]
Frame[6048] processed time[0.10]
Frame[6049] processed time[0.09]
Frame[6050] processed time[0.10]
Frame[6051] processed time[0.10]
Frame[6052] processed time[0.12]
Frame[6053] processed time[0.10]
Frame[6054] processed time[0.11]
Frame[6055] processed time[0.10]
Frame[6056

Frame[6275] processed time[0.12]
Frame[6276] processed time[0.10]
Frame[6277] processed time[0.10]
Frame[6278] processed time[0.12]
Frame[6279] processed time[0.11]
Frame[6280] processed time[0.12]
Frame[6281] processed time[0.10]
Frame[6282] processed time[0.11]
Frame[6283] processed time[0.10]
Frame[6284] processed time[0.13]
Frame[6285] processed time[0.10]
Frame[6286] processed time[0.10]
Frame[6287] processed time[0.09]
Frame[6288] processed time[0.10]
Frame[6289] processed time[0.11]
Frame[6290] processed time[0.10]
Frame[6291] processed time[0.10]
Frame[6292] processed time[0.09]
Frame[6293] processed time[0.10]
Frame[6294] processed time[0.11]
Frame[6295] processed time[0.10]
Frame[6296] processed time[0.12]
Frame[6297] processed time[0.10]
Frame[6298] processed time[0.12]
Frame[6299] processed time[0.10]
Frame[6300] processed time[0.12]
Frame[6301] processed time[0.11]
Frame[6302] processed time[0.12]
Frame[6303] processed time[0.12]
Frame[6304] processed time[0.12]
Frame[6305

Frame[6524] processed time[0.10]
Frame[6525] processed time[0.10]
Frame[6526] processed time[0.11]
Frame[6527] processed time[0.09]
Frame[6528] processed time[0.10]
Frame[6529] processed time[0.11]
Frame[6530] processed time[0.11]
Frame[6531] processed time[0.12]
Frame[6532] processed time[0.11]
Frame[6533] processed time[0.12]
Frame[6534] processed time[0.12]
Frame[6535] processed time[0.12]
Frame[6536] processed time[0.10]
Frame[6537] processed time[0.12]
Frame[6538] processed time[0.11]
Frame[6539] processed time[0.12]
Frame[6540] processed time[0.11]
Frame[6541] processed time[0.11]
Frame[6542] processed time[0.10]
Frame[6543] processed time[0.10]
Frame[6544] processed time[0.10]
Frame[6545] processed time[0.11]
Frame[6546] processed time[0.12]
Frame[6547] processed time[0.10]
Frame[6548] processed time[0.11]
Frame[6549] processed time[0.10]
Frame[6550] processed time[0.12]
Frame[6551] processed time[0.09]
Frame[6552] processed time[0.11]
Frame[6553] processed time[0.09]
Frame[6554

Frame[6774] processed time[0.12]
Frame[6775] processed time[0.10]
Frame[6776] processed time[0.11]
Frame[6777] processed time[0.10]
Frame[6778] processed time[0.11]
Frame[6779] processed time[0.10]
Frame[6780] processed time[0.12]
Frame[6781] processed time[0.08]
Frame[6782] processed time[0.14]
Frame[6783] processed time[0.11]
Frame[6784] processed time[0.13]
Frame[6785] processed time[0.11]
Frame[6786] processed time[0.12]
Frame[6787] processed time[0.12]
Frame[6788] processed time[0.12]
Frame[6789] processed time[0.10]
Frame[6790] processed time[0.12]
Frame[6791] processed time[0.11]
Frame[6792] processed time[0.11]
Frame[6793] processed time[0.09]
Frame[6794] processed time[0.13]
Frame[6795] processed time[0.09]
Frame[6796] processed time[0.12]
Frame[6797] processed time[0.10]
Frame[6798] processed time[0.10]
Frame[6799] processed time[0.09]
Frame[6800] processed time[0.09]
Frame[6801] processed time[0.10]
Frame[6802] processed time[0.10]
Frame[6803] processed time[0.12]
Frame[6804

Frame[7024] processed time[0.11]
Frame[7025] processed time[0.11]
Frame[7026] processed time[0.12]
Frame[7027] processed time[0.12]
Frame[7028] processed time[0.12]
Frame[7029] processed time[0.12]
Frame[7030] processed time[0.12]
Frame[7031] processed time[0.11]
Frame[7032] processed time[0.13]
Frame[7033] processed time[0.10]
Frame[7034] processed time[0.14]
Frame[7035] processed time[0.13]
Frame[7036] processed time[0.12]
Frame[7037] processed time[0.12]
Frame[7038] processed time[0.13]
Frame[7039] processed time[0.11]
Frame[7040] processed time[0.12]
Frame[7041] processed time[0.12]
Frame[7042] processed time[0.12]
Frame[7043] processed time[0.11]
Frame[7044] processed time[0.12]
Frame[7045] processed time[0.10]
Frame[7046] processed time[0.12]
Frame[7047] processed time[0.12]
Frame[7048] processed time[0.12]
Frame[7049] processed time[0.11]
Frame[7050] processed time[0.12]
Frame[7051] processed time[0.11]
Frame[7052] processed time[0.13]
Frame[7053] processed time[0.11]
Frame[7054